In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from shapely.ops import split, GeometryCollection, Point,MultiPoint,LineString
from pyproj import Transformer
import os
import geopandas as gpd

In [3]:
root_name = 'Results'

In [4]:
def change_crs(df,fr,to):
    # Transformer for EPSG 4326 to EPSG 3857
    transformer = Transformer.from_crs(f"epsg:{fr}",f"epsg:{to}", always_xy=True)

    # Function to apply transformation
    def transform_coords(x, y):
        return transformer.transform(y, x)

    # Apply transformation to the columns
    df[['left_lat', 'left_long']] = df.apply(lambda row: transform_coords(row['left_lat'], row['left_long']), axis=1, result_type='expand')
    df[['right_lat', 'right_long']] = df.apply(lambda row: transform_coords(row['right_lat'], row['right_long']), axis=1, result_type='expand')
    # df[['mid_point_x', 'mid_point_y']] = df.apply(lambda row: transform_coords(row['mid_point_x'], row['mid_point_y']), axis=1, result_type='expand')

    return df

In [5]:
os.listdir(root_name)[0].split('_')[2].split('.')[0]

'full circuit'

In [6]:
names = []
direction = []
variations=[]
dem_res = []
left =[]
right= []
average = []
for i in os.listdir(root_name):
    pth = os.path.join(root_name,i)
    df = pd.read_csv(pth)
    new_df = change_crs(df,4326,27700)
    left_length =LineString([i for i in zip(new_df['left_long'].to_list(),new_df['left_lat'].to_list())]).length/1609.34
    right_length = LineString([i for i in zip(new_df['right_long'].to_list(),new_df['right_lat'].to_list())]).length/1609.34
    # print(f'{i.split('_')[1:3]} left_length={left_length}  right_length= {right_length}')
    names.append(i.split('_')[0])
    direction.append(i.split('_')[1])
    variations.append(i.split('_')[2].split('.')[0])
    dem_res.append(30)
    left.append(left_length)
    right.append(right_length)
    average.append((left_length+right_length)/2)
    print(f'average= {(left_length+right_length)/2}')
    print('***************')

average= 1.7278582508626417
***************
average= 1.1377191191887608
***************
average= 2.2175831941968993
***************
average= 2.0251005177412607
***************
average= 0.7217077234738676
***************
average= 1.176465436550772
***************
average= 3.4300234411958224
***************
average= 2.4546870574587807
***************
average= 2.7979190508074314
***************
average= 0.7209375161875395
***************
average= 2.056778416387401
***************
average= 1.3243585215483493
***************
average= 1.1288803442619315
***************
average= 1.7829179786985656
***************
average= 0.9500359382636646
***************
average= 1.5222702259252878
***************
average= 2.880887208027702
***************
average= 0.842477209023186
***************
average= 0.8305586199063888
***************
average= 1.4328227094729677
***************
average= 1.420517492647733
***************
average= 0.538577194419211
***************
average= 2.658400876576434
***********

In [7]:
pd.DataFrame({'Name':names,'Variation':variations,'Resolution':dem_res,'Direction':direction,'Left_Length':left,'Right_length':right,'Average':average}).to_csv('Circuit_lengths_new.csv',index=False)
# Plotting the lengths